In [1]:
import pandas as pd
from general_preprocess import *

import numpy as np

In [2]:
inter_info = pd.read_csv("../ml-1m/ratings.dat", sep="::", engine="python", encoding="latin-1", names=["user_id:token", "item_id:token", "rating:float", "timestamp:float"])

inter_info.head()

,user_id:token,item_id:token,rating:float,timestamp:float
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
item_info = pd.read_csv("../ml-1m/movies.dat", sep="::", engine="python", encoding="latin-1", index_col=0, names=["item_id:token", "movie_title:token_seq", "class:token_seq"])
item_info[["movie_title:token_seq", "release_year:token"]] = item_info["movie_title:token_seq"].str.rsplit(" ", n=1, expand=True)
item_info["release_year:token"] = item_info["release_year:token"].str[1:-1]

item_preprocess(item_info, inter_info)

item_info.head()

,movie_title:token_seq,class:token_seq,release_year:token,average rating,pos/neg item,total ratings (%),popular item
item_id:token,,,,,,,
1,Toy Story,Animation|Children's|Comedy,1995,4.146846,pos,0.002077,H
2,Jumanji,Adventure|Children's|Fantasy,1995,3.201141,None,0.000701,M
3,Grumpier Old Men,Comedy|Romance,1995,3.016736,None,0.000478,M
4,Waiting to Exhale,Comedy|Drama,1995,2.729412,neg,0.000170,T
5,Father of the Bride Part II,Comedy,1995,3.006757,None,0.000296,M


In [4]:
user_info = pd.read_csv("../ml-1m/users.dat", sep="::", engine="python", index_col=0, names=["user_id:token", "gender:token", "age:token", "occupation:token", "zip_code:token"])

user_preprocess(user_info, item_info, inter_info)

user_info.head()

,gender:token,age:token,occupation:token,zip_code:token,items rated,popular items rated (%),mainstream user,mainstream class (thresholds),mainstream class (even groups)
user_id:token,,,,,,,,,
1,F,1,10,48067,53,0.396226,1,3,4
2,M,56,16,70072,129,0.255814,1,2,2
3,M,25,15,55117,51,0.490196,1,4,4
4,M,45,7,02460,21,0.619048,1,4,4
5,M,25,20,55455,198,0.207071,0,2,1


In [13]:
# Distribution of 5-star ratings per user group
# print([[(inter_info.loc[(inter_info["user_id:token"].isin(user_info[user_info["mainstream class (even groups)"] == user_label].index)) & (inter_info["item_id:token"].isin(item_info[item_info["popular item"] == item_label].index))]["rating:float"].values == 5).sum() for item_label in ["H", "M", "T"]] for user_label in range(5)])

# Average amount of items rated by niche and mainstream user groups
# print(user_info.loc[user_info["mainstream class (even groups)"] == 0]["items rated"].mean())
# print(user_info.loc[user_info["mainstream class (even groups)"] == 4]["items rated"].mean())

# Distribution of items based on their popularity group
# print([(item_info["popular item"] == item_label).sum() for item_label in ["H", "M", "T"]])

# Distribution of interactions based on item group
# print([len(inter_info.loc[inter_info["item_id:token"].isin(item_info.loc[item_info["popular item"] == label].index)]) for label in ["H", "M", "T"]])

# The amount of interactions per user
# ints_per_user = [len(inter_info.loc[inter_info["user_id:token"] == user_id]["item_id:token"].values) for user_id in set(inter_info["user_id:token"].values)]
# print(ints_per_user)

# The amount of interactions per item group per user
# ints_per_user_per_item_group = [[(item_info.loc[inter_info.loc[inter_info["user_id:token"] == user_id]["item_id:token"].values]["popular item"].values == item_label).sum() for item_label in ["H", "M", "T"]] for user_id in set(inter_info["user_id:token"].values)]

# Average item rating per item group (and also split per user group)
# print([np.nanmean(item_info.loc[item_info["popular item"] == label]["average rating"].values) for label in ["H", "M", "T"]])
# for u_label, avgs in [(user_label, [inter_info.loc[inter_info["item_id:token"].isin(item_info[item_info["popular item"] == item_label].index)].loc[inter_info["user_id:token"].isin(user_info[user_info["mainstream class (even groups)"] == user_label].index)]["rating:float"].values.mean() for item_label in ["H", "M", "T"]]) for user_label in range(5)]:
#     print(u_label, avgs)

# Distribution of rated item popularity per user group
# item_pop_dist = [sum([np.array(ints_per_user_per_item_group[i]) for i in range(len(ints_per_user_per_item_group)) if user_info.iloc[i]["mainstream class (even groups)"] == user_group]) for user_group in range(5)]
# print([(i, item_pop_dist[i] / sum(item_pop_dist[i]) * 100) for i in range(len(item_pop_dist))])

# Average item popularity distribution across all users
# print(sum([np.array(dist) / sum(dist) for dist in ints_per_user_per_item_group]))

[[21, 29, 3], [33, 86, 10], [25, 23, 3], [13, 8, 0], [41, 99, 58], [13, 46, 12], [20, 11, 0], [28, 87, 24], [38, 63, 5], [79, 264, 58], [34, 81, 22], [7, 14, 2], [39, 60, 9], [11, 12, 2], [46, 139, 16], [5, 20, 10], [56, 121, 34], [52, 214, 39], [71, 147, 37], [13, 10, 1], [6, 13, 3], [61, 200, 36], [61, 207, 36], [51, 78, 7], [33, 43, 9], [53, 245, 102], [18, 46, 6], [43, 55, 9], [40, 55, 13], [17, 16, 10], [20, 73, 26], [16, 25, 7], [70, 250, 71], [36, 112, 16], [25, 121, 52], [85, 236, 30], [13, 32, 8], [17, 63, 20], [24, 32, 6], [19, 57, 20], [9, 14, 2], [66, 153, 12], [12, 11, 1], [59, 101, 33], [29, 169, 99], [1, 16, 24], [11, 9, 2], [95, 406, 97], [35, 64, 9], [7, 21, 15], [15, 18, 7], [20, 51, 8], [84, 414, 186], [10, 25, 5], [15, 9, 1], [25, 40, 2], [13, 32, 19], [91, 306, 40], [37, 134, 42], [14, 46, 10], [4, 21, 11], [66, 300, 132], [23, 59, 16], [11, 15, 1], [44, 72, 5], [10, 11, 5], [8, 48, 8], [24, 40, 8], [13, 36, 16], [24, 26, 4], [10, 18, 1], [15, 24, 4], [60, 157, 38]

In [ ]:
user_info.to_csv("../ml-1m/ml-1m.user", sep="\t")
item_info.to_csv("../ml-1m/ml-1m.item", sep="\t")
inter_info.to_csv("../ml-1m/ml-1m.inter", index=None, sep="\t")